# Game sales analytic
Author: Korobkov Lev DSBA 242-2

**It is highly recommended to view the project in the web interface**

Web interface link:


Below, the entire back will be written first, after which the analysis itself and a description of it will be written.

In [29]:
import pandas as pd
from enum import StrEnum
import plotly.express as px

In [30]:
# Config
class ColNames(StrEnum):
    RANK = "Rank"
    NAME = "Name"
    PLATFORM = "Platform"
    YEAR = "Year"
    GENRE = "Genre"
    PUBLISHER = "Publisher"
    NA_SALES = "NA_Sales"
    EU_SALES = "EU_Sales"
    JP_SALES = "JP_Sales"
    OTHER_SALES = "Other_Sales"
    GLOBAL_SALES = "Global_Sales"
    TOTAL_SALES = "Total_Sales"
    PERCENT = "Percent_of_total_sales"
    SALES = "Game_Sales"
    NA_PERCENT = "NA_Percent"
    EU_PERCENT = "EU_Percent"
    JP_PERCENT = "JP_Percent"
    OTHER_PERCENT = "Other_Percent"
    PERCENTAGE = "Percentage"
    TOTAL_COUNT = "Total_Count"
    REGION = "Region"


genres_present = [
    "Sports",
    "Platform",
    "Racing",
    "Role-Playing",
    "Puzzle",
    "Misc",
    "Shooter",
    "Simulation",
    "Action",
    "Fighting",
    "Adventure",
    "Strategy",
]


platforms_present = [
    "DS",
    "N64",
    "PS3",
    "XB",
    "GC",
    "NG",
    "SAT",
    "GG",
    "NES",
    "PSV",
    "3DO",
    "X360",
    "GB",
    "GEN",
    "2600",
    "PS",
    "Wii",
    "TG16",
    "WS",
    "GBA",
    "WiiU",
    "3DS",
    "PCFX",
    "PS4",
    "PS2",
    "XOne",
    "DC",
    "SCD",
    "PSP",
    "SNES",
    "PC",
]

In [31]:
# Services
def create_dataframe() -> pd.DataFrame:
    dataframe = pd.read_csv("./kaggle/vgsales.csv")
    dataframe = dataframe.dropna().reset_index(drop=True)
    return dataframe


def count_total_sales(df: pd.DataFrame) -> pd.DataFrame:
    total_sales = dict()
    for i in range(len(df)):
        if df.loc[i][ColNames.NAME] in total_sales:
            total_sales[df.loc[i][ColNames.NAME]] += float(
                df.loc[i][ColNames.GLOBAL_SALES]
            )
        else:
            total_sales[df.loc[i][ColNames.NAME]] = float(
                df.loc[i][ColNames.GLOBAL_SALES]
            )
    for name, sales in total_sales.items():
        total_sales[name] = round(float(sales), 5)
    df[ColNames.TOTAL_SALES] = None
    for i in range(len(df)):
        df.at[i, ColNames.TOTAL_SALES] = total_sales[df.loc[i][ColNames.NAME]]
    return df


def create_total_sales_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    total_sales = dict()
    for i in range(len(df)):
        if df.loc[i][ColNames.NAME] in total_sales:
            total_sales[df.loc[i][ColNames.NAME]] += float(
                df.loc[i][ColNames.GLOBAL_SALES]
            )
        else:
            total_sales[df.loc[i][ColNames.NAME]] = float(
                df.loc[i][ColNames.GLOBAL_SALES]
            )
    dataframe = pd.DataFrame(
        columns=[ColNames.NAME, ColNames.PUBLISHER, ColNames.TOTAL_SALES]
    )
    i = 0
    for name, sales in total_sales.items():
        total_sales[name] = round(float(sales), 5)
        dataframe.at[i, ColNames.NAME] = name
        dataframe.at[i, ColNames.TOTAL_SALES] = sales
        dataframe.at[i, ColNames.PUBLISHER] = df.loc[df[ColNames.NAME] == name].iloc[0][
            ColNames.PUBLISHER
        ]
        dataframe.at[i, ColNames.TOTAL_SALES] = sales
        i += 1
    return dataframe


def game_platform_sales_percentage(
    df: pd.DataFrame, game: str, platform: str
) -> list[float, float]:
    if platform in platforms_present:
        for i in range(len(df)):
            if (
                df.loc[i][ColNames.PLATFORM] == platform
                and df.loc[i][ColNames.NAME] == game
            ):
                return [
                    round(
                        float(df.loc[i][ColNames.GLOBAL_SALES])
                        / float(df.loc[i][ColNames.TOTAL_SALES])
                        * 100,
                        2,
                    ),
                    float(df.loc[i][ColNames.GLOBAL_SALES]),
                ]


def platform_sales(df: pd.DataFrame) -> pd.DataFrame:
    platform_amount = dict()
    all_sales_amount = 0
    for i in range(len(df)):
        if df.loc[i][ColNames.PLATFORM] in platform_amount:
            platform_amount[df.loc[i][ColNames.PLATFORM]] += float(
                df.loc[i][ColNames.GLOBAL_SALES]
            )
        else:
            platform_amount[df.loc[i][ColNames.PLATFORM]] = float(
                df.loc[i][ColNames.GLOBAL_SALES]
            )
        all_sales_amount += float(df.loc[i][ColNames.GLOBAL_SALES])
    all_sales_amount = round(all_sales_amount, 3)
    platform_sales = pd.DataFrame(
        columns=[ColNames.PLATFORM, ColNames.SALES, ColNames.PERCENT]
    )
    for name, sales in platform_amount.items():
        platform_sales.loc[len(platform_sales)] = [
            name,
            round(sales, 3),
            round(sales / all_sales_amount * 100, 4),
        ]
    return platform_sales


def create_sales_percentage_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df[ColNames.NA_PERCENT] = None
    df[ColNames.EU_PERCENT] = None
    df[ColNames.JP_PERCENT] = None
    df[ColNames.OTHER_PERCENT] = None
    for i in range(len(df)):
        df.at[i, ColNames.NA_PERCENT] = round(
            float(df.loc[i][ColNames.NA_SALES])
            / float(df.loc[i][ColNames.GLOBAL_SALES])
            * 100,
            3,
        )
        df.at[i, ColNames.EU_PERCENT] = round(
            float(df.loc[i][ColNames.EU_SALES])
            / float(df.loc[i][ColNames.GLOBAL_SALES])
            * 100,
            3,
        )
        df.at[i, ColNames.JP_PERCENT] = round(
            float(df.loc[i][ColNames.JP_SALES])
            / float(df.loc[i][ColNames.GLOBAL_SALES])
            * 100,
            3,
        )
        df.at[i, ColNames.OTHER_PERCENT] = round(
            float(df.loc[i][ColNames.OTHER_SALES])
            / float(df.loc[i][ColNames.GLOBAL_SALES])
            * 100,
            3,
        )
    return df


def create_first_tsd_presentation_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df = df.groupby(ColNames.PUBLISHER, as_index=False)[ColNames.TOTAL_SALES].sum()
    df[ColNames.PERCENTAGE] = (
        df[ColNames.TOTAL_SALES] / df[ColNames.TOTAL_SALES].sum() * 100
    )
    border = 1
    major_publishers = df[df[ColNames.PERCENTAGE] >= border]
    other_publishers = df[df[ColNames.PERCENTAGE] < border]
    other_total = other_publishers[ColNames.TOTAL_SALES].sum()
    other_row = pd.DataFrame(
        {
            ColNames.PUBLISHER: ["Others"],
            ColNames.TOTAL_SALES: [other_total],
            ColNames.PERCENTAGE: [other_total / df[ColNames.TOTAL_SALES].sum() * 100],
        }
    )
    tsd_presentation_dataframe = pd.concat(
        [major_publishers, other_row], ignore_index=True
    )
    return tsd_presentation_dataframe


def create_games_by_publisher_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    count_games = df.groupby(ColNames.PUBLISHER)[ColNames.NAME].count().reset_index()
    count_games.columns = [ColNames.PUBLISHER, ColNames.TOTAL_COUNT]
    return count_games


def create_second_tsd_presentation_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df = df.groupby(ColNames.PUBLISHER, as_index=False)[ColNames.TOTAL_COUNT].sum()
    df[ColNames.PERCENTAGE] = (
        df[ColNames.TOTAL_COUNT] / df[ColNames.TOTAL_COUNT].sum() * 100
    )
    border = 1
    major_publishers = df[df[ColNames.PERCENTAGE] >= border]
    other_publishers = df[df[ColNames.PERCENTAGE] < border]
    other_total = other_publishers[ColNames.TOTAL_COUNT].sum()
    other_row = pd.DataFrame(
        {
            ColNames.PUBLISHER: ["Others"],
            ColNames.TOTAL_COUNT: [other_total],
            ColNames.PERCENTAGE: [other_total / df[ColNames.TOTAL_COUNT].sum() * 100],
        }
    )
    tsd_presentation_dataframe = pd.concat(
        [major_publishers, other_row], ignore_index=True
    )
    return tsd_presentation_dataframe


def spd_dataframes(df: pd.DataFrame) -> pd.DataFrame:
    spd_dataframe = pd.DataFrame(
        columns=[ColNames.GENRE, ColNames.YEAR, ColNames.SALES]
    )
    for i in range(len(df)):
        spd_dataframe.at[i, ColNames.GENRE] = df.iloc[i][ColNames.GENRE]
        spd_dataframe.at[i, ColNames.YEAR] = df.iloc[i][ColNames.YEAR]
        spd_dataframe.at[i, ColNames.SALES] = df.iloc[i][ColNames.GLOBAL_SALES]
    spd_first_dataframe = spd_dataframe.groupby([ColNames.GENRE], as_index=False)[
        ColNames.SALES
    ].sum()
    spd_second_dataframe = spd_dataframe.groupby(
        [ColNames.GENRE, ColNames.YEAR], as_index=False
    )[ColNames.SALES].sum()
    return [spd_first_dataframe, spd_second_dataframe]


def spd_year_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    return df.groupby([ColNames.YEAR], as_index=False)[ColNames.GLOBAL_SALES].sum()


def spd_region_sales_dataframe(df: pd.DataFrame, platform: str) -> pd.DataFrame:
    df = (
        df.groupby(ColNames.PLATFORM)[
            [
                ColNames.NA_PERCENT,
                ColNames.EU_PERCENT,
                ColNames.JP_PERCENT,
                ColNames.OTHER_PERCENT,
            ]
        ]
        .mean()
        .reset_index()
    )
    df = df[df[ColNames.PLATFORM] == platform]
    sales_summary = df[
        [
            ColNames.NA_PERCENT,
            ColNames.EU_PERCENT,
            ColNames.JP_PERCENT,
            ColNames.OTHER_PERCENT,
        ]
    ].sum()
    spd_region = pd.DataFrame(
        {
            ColNames.REGION: [
                ColNames.NA_PERCENT,
                ColNames.EU_PERCENT,
                ColNames.JP_PERCENT,
                ColNames.OTHER_PERCENT,
            ],
            ColNames.SALES: [
                sales_summary[ColNames.NA_PERCENT],
                sales_summary[ColNames.EU_PERCENT],
                sales_summary[ColNames.JP_PERCENT],
                sales_summary[ColNames.OTHER_PERCENT],
            ],
        }
    )
    return spd_region


def hypothesis(df: pd.DataFrame) -> pd.DataFrame:
    df = df.groupby([ColNames.NAME, ColNames.GENRE], as_index=False)[
        ColNames.GLOBAL_SALES
    ].sum()
    df = df[df[ColNames.GLOBAL_SALES] > 10]
    hypothesis_dataframe = (
        df.groupby(ColNames.GENRE).size().reset_index(name=ColNames.TOTAL_COUNT)
    )
    return hypothesis_dataframe

In [32]:
# Call
original_dataframe = create_dataframe()
main_dataframe = count_total_sales(original_dataframe)
sales_percentage_dataframe = create_sales_percentage_dataframe(original_dataframe)
total_sales_dataframe = create_total_sales_dataframe(main_dataframe)
count_games_by_publisher = create_games_by_publisher_dataframe(total_sales_dataframe)

# Overview

In [33]:
original_dataframe

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Total_Sales,NA_Percent,EU_Percent,JP_Percent,Other_Percent
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,82.74,50.145,35.074,4.556,10.225
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,45.31,72.266,8.897,16.923,1.914
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,35.82,44.249,35.958,10.581,9.241
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,33.0,47.727,33.364,9.939,8.97
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,31.37,35.926,28.339,32.579,3.188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16286,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01,0.01,100.0,0.0,0.0,0.0
16287,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01,0.51,100.0,0.0,0.0,0.0
16288,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01,0.31,0.0,0.0,0.0,0.0
16289,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01,0.01,0.0,100.0,0.0,0.0


# Original kaggle dataframe above

Our dataframe contains data about games with sales greater than 100,000 copies: their sales, on which platforms they were sold, by which publishers, year of release and in which markets how many were sold.

Let's modify it to understand what percentage of the game's sales were in a particular market and how many copies of each game were sold in total from all platforms.

In [34]:
sales_percentage_dataframe

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Total_Sales,NA_Percent,EU_Percent,JP_Percent,Other_Percent
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,82.74,50.145,35.074,4.556,10.225
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,45.31,72.266,8.897,16.923,1.914
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,35.82,44.249,35.958,10.581,9.241
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,33.0,47.727,33.364,9.939,8.97
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,31.37,35.926,28.339,32.579,3.188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16286,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01,0.01,100.0,0.0,0.0,0.0
16287,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01,0.51,100.0,0.0,0.0,0.0
16288,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01,0.31,0.0,0.0,0.0,0.0
16289,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01,0.01,0.0,100.0,0.0,0.0


# Modificated dataframe for overview above

We have added data on the percentage of game sales in different regions. Now we will look at the various data and analyze it.

In [35]:
spd_dataframes_ = spd_dataframes(original_dataframe)
spd_presentation_fig = px.bar(
    spd_dataframes_[0],
    x=ColNames.GENRE,
    y=ColNames.SALES,
    title='Sales by genre',
)
spd_presentation_fig.show()

As we can see, the best-selling games are games in the Action and Sports genres, the most unsold are Strategies.

In [36]:
spd_presentation_fig = px.bar(
    spd_year_dataframe(original_dataframe),
    x=ColNames.YEAR,
    y=ColNames.GLOBAL_SALES,
    title='Sales per year'
)
spd_presentation_fig.show()

Based on this bar chart, we see that the dataset does not contain enough information about 2016-2024. Excluding this fact, we are seeing a decrease in sales of games with more than 100,000 copies, therefore, since 2010, the number of sales of small indie projects that do not collect 100,000 copies has been growing, and large games that collect huge sales have become much less frequent.

Next, we can look at the trend of sales of games in genres by year and you can draw the obvious conclusion yourself: the popularity of the genre is decreasing or increasing

In [37]:
spd_presentation_fig = px.line(
    spd_dataframes_[1][spd_dataframes_[1][ColNames.GENRE] == 'Action'],
    x=ColNames.YEAR,
    y=ColNames.SALES,
    title=f'Sales of Action per year',
)
spd_presentation_fig.show()
spd_presentation_fig = px.line(
    spd_dataframes_[1][spd_dataframes_[1][ColNames.GENRE] == 'Shooter'],
    x=ColNames.YEAR,
    y=ColNames.SALES,
    title=f'Sales of Shooter per year',
)
spd_presentation_fig.show()
spd_presentation_fig = px.line(
    spd_dataframes_[1][spd_dataframes_[1][ColNames.GENRE] == 'Platform'],
    x=ColNames.YEAR,
    y=ColNames.SALES,
    title=f'Sales of Platformers per year',
)
spd_presentation_fig.show()

Here we notice an important point: the dataset does not contain enough information about 2016-2024, so a strong drawdown is noticed in all genres closer to our time. Therefore, we can make an unambiguous statement: all the analysis will be performed based on old data, and therefore something could have changed in our time.

You can also check other genres yourself and draw similar conclusions using the Web Interface.

Now let's look at the average percentage of video game sales by region on specific platforms.

In [38]:
first = px.pie(
    spd_region_sales_dataframe(sales_percentage_dataframe,'PC'),
    values=ColNames.SALES,
    names=ColNames.REGION,
    title=f"PC", 
)
first.show()
second = px.pie(
    spd_region_sales_dataframe(sales_percentage_dataframe,'XOne'),
    values=ColNames.SALES,
    names=ColNames.REGION,
    title=f"Xbox One ", 
)
second.show()
third = px.pie(
    spd_region_sales_dataframe(sales_percentage_dataframe,'PS3'),
    values=ColNames.SALES,
    names=ColNames.REGION,
    title=f"Playstation 3", 
)
third.show()

**Percentages of PC, Xbox One and Playstation 3 sales by region**

Following from these pie charts, it can be concluded that if a developer wants to sell his game mostly on the European market, he needs to develop it for the PC. If he wants to sell the game in America and Europe, his choice is Xbox. However, in order to sell the game worldwide, he needs to choose a Playstation.

You can also check other platforms yourself and draw similar conclusions using the Web Interface.

# Deeper Overview

In [39]:
tsd_presentation_fig = px.pie(
    create_first_tsd_presentation_dataframe(total_sales_dataframe),
    values=ColNames.TOTAL_SALES,
    names=ColNames.PUBLISHER,
    title="Publishers shares in the market", 
)
tsd_presentation_fig.show()

*This pie chart puts all publishers with less than 1% of global sales in the "Others" category.*

As we can see, 17.2% of global sales are from small publishers, while there are also hegemons such as Nintendo, whose sales account for 20.1% of global sales. Considering that there are a large number of companies in the video game market, it is estimated that the 5 largest publishers (*Nintendo, Electronic Arts, Activison, Sony Computer Entertainment and Ubisoft*) occupy the most number of players - 52.74%'

*Now let's look at the number of games released by different companies*

In [40]:
tsd_presentation_fig = px.pie(
    create_second_tsd_presentation_dataframe(count_games_by_publisher),
    values=ColNames.TOTAL_COUNT,
    names=ColNames.PUBLISHER,
    title="Number of games by publisher", 
)
tsd_presentation_fig.show()

Now we are witnessing the fact that the number of games released does not always lead to more sales. So Namco Bandai Games have made the most games among the major publishers, however (from the past pie chart) we see that their sales are 2.83%.'

It is also noticeable that independent (small) publishers release much more games (38.3% of all time) and at the same time their sales account for only 17.2% of the global total.'

In general, the following conclusion can be drawn from this: it is much more profitable to sell games through large publishers than through your own or small publishing companies. It also means that for a small company, the threshold for entering the video game market is quite high.'

# Hypothesis

**Hypothesis:** Games with global sales over 10 million are more likely to belong to the "Action" or "Shooter" genres than any other genres.',"*Let's check if this is true.*

In [41]:
hypothesis_presentation_fig = px.pie(
    hypothesis(main_dataframe),
    values=ColNames.TOTAL_COUNT,
    names=ColNames.GENRE,
    title="Number of games with global sales over 10 million by genre", 
)
hypothesis_presentation_fig.show()

The hypothesis is partially correct:** "Action" and "Shooter" are the most popular genres among over 10 million global sales games, but their dominance is not overwhelming compared to other genres: 34.4% of "Action"+"Shooter" againts 65.6% of other genres. Other genres, like "Role-Playing," "Sports," and "Platform," also contribute substantially to the list of games with over 10 million global sales.

**Thank you for your attention!**